# Logging within Deltaflow

It is helpful when debugging to be able to log intermediate computations. In this notebook, we demonstrate the different levels and ways in which one can log messages in Deltaflow.

In [1]:
import logging

from deltalanguage.lib import make_generator, StateSaver
from deltalanguage.logging import make_logger
from deltalanguage.runtime import DeltaPySimulator, DeltaRuntimeExit
from deltalanguage.wiring import (DeltaBlock, DeltaGraph,
                                  Interactive, PyInteractiveNode)

## Runtime logging

The first way we can log messages is at the Python runtime level.
At this level, we can log the creation of queues and threads, and log each tick of the main thread.
When the program terminates, or when an error occurs in a node, this is logged as well.

To demonstrate this, below we create a simple graph, consisting of one node which adds two integers together and another which saves the result before exiting the program.

In [2]:
@DeltaBlock(allow_const=False)
def add(a: int, b: int) -> int:
    return a + b

s = StateSaver(int, verbose=True)

with DeltaGraph() as graph_1:
    s.save_and_exit(add(2, 3))

When creating the `DeltaPySimulator`, we can use the `lvl` parameter to specify the level at which we display logged messages. This can be any integer, but the most common are `logging.DEBUG` (10), `logging.INFO` (20), `logging.WARNING` (30) and `logging.ERROR` (40). Note that messages logged at higher levels are reported, so a log set at `logging.INFO` will include messages from `logging.WARNING` and `logging.ERROR`. See Python's [logging levels](https://docs.python.org/3/library/logging.html#logging-levels) for further details.

In the example below, we will set our runtime logging level to `logging.INFO`.

In [3]:
rt = DeltaPySimulator(graph_1, lvl=logging.INFO)
rt.run()

assert s.saved == [5]

DeltaPySimulator [INFO]: creating queue: node_0________________________ to add_2.a_______________________ DInt32______________
DeltaPySimulator [INFO]: creating queue: node_1________________________ to add_2.b_______________________ DInt32______________
DeltaPySimulator [INFO]: creating queue: add_2_________________________ to save_and_exit_3.val___________ DInt32______________
DeltaPySimulator [INFO]: Starting node add_2
DeltaPySimulator [INFO]: Starting node save_and_exit_3
DeltaPySimulator [INFO]: Thread stopped: _thread.ExceptHookArgs(exc_type=<class 'deltalanguage.runtime._runtime.DeltaRuntimeExit'>, exc_value=DeltaRuntimeExit(), exc_traceback=<traceback object at 0x7f3e06fba540>, thread=<DeltaThread(Thread_save_and_exit_3, started 139904381761280)>)
DeltaPySimulator [INFO]: Total number of threads = 3
DeltaPySimulator [INFO]: ========== TICK 1 ==========
DeltaPySimulator [INFO]: Number of active threads = 6
DeltaPySimulator [INFO]: Thread stopped: _thread.ExceptHookArgs(exc_type=

## Logging within an individual block

We can also record computation performed by a single `DeltaBlock`. To do this, we specify the `lvl` parameter in the decorator for a node.

In the example below, we create a new node which adds two integers together, but this time we set the logging level to `logging.DEBUG`.

In [4]:
@DeltaBlock(allow_const=False, lvl=logging.DEBUG)
def add_log(a: int, b: int) -> int:
    return a + b

We can wire up this node the same way as before and see that this time we log what messages we send to the `save_and_exit` node.

In [5]:
s = StateSaver(int, verbose=True)

with DeltaGraph() as graph_2:
    s.save_and_exit(add_log(5, 6))

rt = DeltaPySimulator(graph_2)
rt.run()

assert s.saved == [11]

PyFuncNode add_log_6 [DEBUG]: Running...
PyFuncNode add_log_6 [DEBUG]: Running...
PyFuncNode add_log_6 [INFO]: Stopped add_log_6.
saving 11


This same parameter can also be used to log computation in other types of nodes, such as method nodes, interactive nodes and even Migen nodes.

## Logging within a node body

In this next example, we are going to create our own logger and use it within a node's body.

Below we have an interactive node, which will count down and logs each integer it sends.

To log what is happening in this node, we use the `make_logger` command, which will create a log for us with a given level and name. This is the same command used for logging throughout Deltaflow.

In [6]:
@Interactive(in_params={}, out_type=int)
def count_and_log(node: PyInteractiveNode):
    logger = make_logger(lvl=logging.DEBUG, name="count_and_log")
    for i in range(10, -1, -1):
        logger.info(f"i= {i}")
        node.send(i)

We are going to wire this node up to a state saver node again, and this time exit the program once we reach 0.

In [7]:
s = StateSaver(int, condition=lambda x: x == 0, verbose=True)

with DeltaGraph() as graph_3:
    s.save_and_exit_if(count_and_log.call())

Running this below, we can see the logs from our custom logger.

In [8]:
rt = DeltaPySimulator(graph_3)
rt.run()

assert s.saved == [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

count_and_log [INFO]: i= 10
count_and_log [INFO]: i= 9
count_and_log [INFO]: i= 8
count_and_log [INFO]: i= 7
count_and_log [INFO]: i= 6
count_and_log [INFO]: i= 5
count_and_log [INFO]: i= 4
count_and_log [INFO]: i= 3
count_and_log [INFO]: i= 2
count_and_log [INFO]: i= 1
count_and_log [INFO]: i= 0
saving 10
saving 9
saving 8
saving 7
saving 6
saving 5
saving 4
saving 3
saving 2
saving 1
saving 0


Note that passing the same name to `make_logger` will return the same logger object. This means that other nodes which run multiple times, such as those created by `DeltaBlock` or `DeltaMethodBlock` can also use `make_logger` to get the same logger each time they're evaluated. Note however that this also means if two nodes request a logger with the same name, they will both get the same logger.

## Logging of Messages

The final case we are going to consider is logging messages that are sent between nodes.

For this, we will use a generator, which creates a Deltaflow node for sending each item in a list of integers to the next node.

In [9]:
sender = make_generator([5, 4, 3, 2, 1, 0])

s = StateSaver(int, condition=lambda x: x == 0, verbose=True)

with DeltaGraph() as graph_4:
    s.save_and_exit_if(sender.call())

The `msg_lvl` parameter in `DeltaPySimulator` is used to determine which level logs from messages are displayed at.

The messages are logged when the program exits, and are sorted by logical clock order, as an aproximation for what time each message was sent.

In [10]:
rt = DeltaPySimulator(graph_4, msg_lvl=logging.INFO)
rt.run()

assert s.saved == [5, 4, 3, 2, 1, 0]

MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 1: 5
MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 2: 4
MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 3: 3
MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 4: 2
MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 5: 1
MessageLog [INFO]: Sent to save_and_exit_if_11 port val at logical clock time 6: 0
saving 5
saving 4
saving 3
saving 2
saving 1
saving 0
